In [1]:
import time
from datasets import DatasetDict, Dataset
import json
import torch
from transformers import T5Tokenizer, DataCollatorForSeq2Seq
from transformers import T5ForConditionalGeneration, Seq2SeqTrainingArguments, Seq2SeqTrainer
import nltk
import evaluate
import numpy as np

In [2]:
# check GPU
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

Using device: cuda


In [3]:
# Load the TLQA data
def read_QA_data(filepath):
    with open(filepath, 'r', encoding='utf-8') as f:
        data = json.load(f)
    return data

In [4]:
# Filter data to only include 'question' and 'answers'
def filter_data(data):
    filtered_data = [{"question": item["question"], "answers": item["answers"]} for item in data]
    return filtered_data

In [5]:
train_data = filter_data(read_QA_data('train_TLQA.json'))
val_data = filter_data(read_QA_data('val_TLQA.json'))
test_data = filter_data(read_QA_data('test_TLQA.json'))

In [6]:
# Print sample data
print("Sample training data:", train_data[0])
print("Sample validation data:", val_data[0])
print("Sample test data:", test_data[0])

Sample training data: {'question': 'List all entities that owned Radisson Hotel Group, also known as Radisson Hospitality, Inc., from 2010 to 2020.', 'answers': ['Carlson (2010, 2011, 2012, 2013, 2014, 2015, 2016)', 'HNA Group (2016, 2017, 2018)', 'Jinjiang International (2018, 2019, 2020)']}
Sample validation data: {'question': 'List all chairpersons of Christian Democratic and Flemish, also known as CD&V, from 2010 to 2020', 'answers': ['Wouter Beke (2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019)', 'Joachim Coens (2019, 2020)']}
Sample test data: {'question': 'List all sports teams Anthony Grant, also known as Anthony Paul Shaun Andrew Daure Grant, played for from 2010 to 2020.', 'answers': ['Southend United F.C. (2010, 2011, 2012)', 'Stevenage F.C. (2012, 2013)', 'Crewe Alexandra F.C. (2013, 2014, 2015)', 'Port Vale F.C. (2015, 2016, 2017, 2018, 2019, 2020)']}


In [7]:
# Convert list of dictionaries to dictionary of lists
def convert_to_dict(data):
    keys = data[0].keys()
    return {key: [d[key] for d in data] for key in keys}

In [8]:
# Convert data to Hugging Face Dataset
train_dataset = Dataset.from_dict(convert_to_dict(train_data))
val_dataset = Dataset.from_dict(convert_to_dict(val_data))
test_dataset = Dataset.from_dict(convert_to_dict(test_data))

In [9]:
# Combine into a DatasetDict
dataset = DatasetDict({
    'train': train_dataset,
    'validation': val_dataset,
    'test': test_dataset
})
dataset

DatasetDict({
    train: Dataset({
        features: ['question', 'answers'],
        num_rows: 3212
    })
    validation: Dataset({
        features: ['question', 'answers'],
        num_rows: 1428
    })
    test: Dataset({
        features: ['question', 'answers'],
        num_rows: 1071
    })
})

In [10]:
pip install transformers[torch]

Note: you may need to restart the kernel to use updated packages.


In [18]:
from transformers import T5ForConditionalGeneration, T5Tokenizer

last_checkpoint = "./results/checkpoint-1500"

#finetuned_model = T5ForConditionalGeneration.from_pretrained(last_checkpoint)
#tokenizer = T5Tokenizer.from_pretrained(last_checkpoint)

# 加上few-shot

In [19]:
def knn_search(train_questions, train_answers, test_questions, k=5, model_name='sentence-transformers/all-mpnet-base-v2'):
  
    model = SentenceTransformer(model_name)
    
   
    train_embeddings = model.encode(train_questions, convert_to_tensor=True).cpu().numpy()
    
    test_embeddings = model.encode(test_questions, convert_to_tensor=True).cpu().numpy()
    
    knn = NearestNeighbors(n_neighbors=k, algorithm='auto').fit(train_embeddings)
    
    few_shot_examples = []
    for test_embedding in test_embeddings:
        distances, indices = knn.kneighbors([test_embedding])
        few_shot_samples = [{'question': train_questions[idx], 'answers': train_answers[idx]} for idx in indices[0]]
        few_shot_examples.append(few_shot_samples)
    
    return few_shot_examples

In [20]:
# Run generative models
def run_model(last_checkpoint, test_questions, few_shot_examples):
    model = T5ForConditionalGeneration.from_pretrained(last_checkpoint)
    tokenizer = T5Tokenizer.from_pretrained(last_checkpoint)
    
    predictions = []
    for test_question, examples in zip(test_questions, few_shot_examples):
        context = "\n".join([f"Q: {ex['question']} A: {', '.join(ex['answers'])}" for ex in examples])
        input_text = f"{context}\nQ: {test_question} A:"
        inputs = tokenizer(input_text, return_tensors="pt", truncation=True, max_length=512)
    
        # Generate prediction with beam search to reduce repetition
        #outputs = model.generate(**inputs, num_beams=5, max_new_tokens = 100, no_repeat_ngram_size=5)
        outputs = model.generate(**inputs, max_new_tokens=100, repetition_penalty=3.0)
    
    
        prediction = tokenizer.decode(outputs[0], skip_special_tokens=True)
    
        predictions.append(prediction)
        print(f"Question: {test_question}\nGenerated Answer: {prediction}\n")


    return predictions

In [21]:
train_data = dataset['train'] 
test_data = dataset['test']
train_questions = train_data["question"]
train_answers = train_data["answers"]
test_questions = test_data["question"]

In [22]:
from sentence_transformers import SentenceTransformer,util
from sklearn.neighbors import NearestNeighbors
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

In [23]:
#examples_3shot  = knn_search(train_questions, train_answers, test_questions, 3)

In [31]:
examples_3shot = read_QA_data('examples_3shot.json')
predictions_3shot = run_model(last_checkpoint, test_questions, examples_3shot)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Question: List all sports teams Anthony Grant, also known as Anthony Paul Shaun Andrew Daure Grant, played for from 2010 to 2020.
Generated Answer: England national under-21 football team (2010), Manchester City F.C. (2010, 2011, 2012), Tottenham Hotspur A.F.C. (2012), West Bromwich Albion F.C. (2013), Stoke City F.C. (2014), Fulham F.C. (2015), Burnley F.C. (2016), Crystal Palace FC (2017, 2018, 2019, 2020)

Question: List all positions Oleksandr Turchynov, also known as Oleksandr Valentynovych Turchynov, held from 2010 to 2020.
Generated Answer: President of Ukraine (2010, 2011, 2012), Prime Minister of Ukraine (2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020)

Question: List all political parties Paul Murphy was a member of from 2010 to 2020.
Generated Answer: Democratic Party (2010, 2011, 2012), independent politician (2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020)

Question: List all sports teams Andy Carroll, also known as Andrew Thomas Carroll, played for from 2010 t

Question: List all sports teams Sam Tomkins played for from 2010 to 2020.
Generated Answer: England national under-21 football team (2010), Manchester City F.C. (2010, 2011, 2012), Tottenham Hotspur A.F.C. (2012), West Bromwich Albion F.C. (2013), Stoke City F.C. (2014), Burnley F.C. (2015), Blackburn Rovers FC (2016), Crewe Alexandra F.C. (2018)

Question: List all sports teams Juan Cuadrado, also known as Juan Guillermo Cuadrado Bello, played for from 2010 to 2020.
Generated Answer: Real Madrid CF (2010, 2011, 2012), FC Barcelona (2010), Spain national association football team (2011, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020)

Question: List all political parties Rachel Carling-Jenkins, also known as Rachel Carling, was a member of from 2010 to 2018.
Generated Answer: Conservative Party (2010, 2011, 2012), Liberal Democrats (2010), National Alliance (2012, 2013, 2014, 2015, 2016, 2017, 2018)

Question: List all coaches of CD Tondela from 2015 to 2020
Generated Answer: tefan Sta

Question: List all positions François Lecointre, also known as General François Lecointre, held from 2016 to 2020.
Generated Answer: President of the French Republic (2016, 2017, 2018, 2019, 2020), Prime Minister of France (2017, 2018)

Question: List all coaches of Vitória S.C., also known as Vitória de Guimâraes, from 2015 to 2020
Generated Answer: José Mário (2015, 2016, 2017, 2018, 2019), Luis Martn Sánchez (2019, 2020)

Question: List all coaches of U.C. Trevigiani Energiapura Marchiol, also known as Unieuro Trevigiani-Hemus 1896, from 2012 to 2020
Generated Answer: Stefano Spalletti (2012, 2013, 2014), Gianfranco Zola (2018), Marco Antonioni (2018, 2019), Maurizio Santi (2019, 2020)

Question: List all coaches of Silkeborg IF, also known as Silkeborg Idrætsforening, from 2015 to 2020
Generated Answer: Erik Srensen (2015, 2016, 2017, 2018, 2019, 2020), Jörg Storck (2015)

Question: List all chairpersons of Roman Catholic Archdiocese of Cardiff, also known as Archdiocese of Cardiff

Question: List all employers Michael Hout, also known as Mike Hout, worked for from 2010 to 2020.
Generated Answer: University of Cambridge (2010, 2011, 2012), Universidade de Barcelona (2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020)

Question: List all heads of the government of Guinea, also known as Guinea-Conakry, from 2010 to 2020 
Generated Answer: Emmanuel N'Alia (2010, 2011, 2012), Ousmane Umaru (2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020)

Question: List all employers Linda Bauld, also known as Linda C. Bauld, worked for from 2014 to 2020.
Generated Answer: University of California, Irvine (2014, 2015), National Institute on Drug Abuse and Narcotics (2015, 2016, 2017, 2018, 2019, 2020)

Question: List all coaches of PSV Eindhoven, also known as Philips Sport Vereniging, from 2013 to 2020
Generated Answer: Mauricio Pochettino (2013, 2014), Marcos Sánchez-Castro (2013), Marcelo De Oliveira (2018), Luis Enrique (2019, 2020)

Question: List all employers Pedro J. R

...

Question: List all positions Stockwell Day, also known as Stockwell Burt Day, held from 2010 to 2011.
Generated Answer: United States representative (2010, 2011), Governor of New York (2011, 2012)

Question: List all coaches of RCD Espanyol de Barcelona, also known as RCDE, from 2010 to 2020
Generated Answer: José Mourinho (2010, 2011, 2012), Luis Enrique (2010), Juan Manuel Pellegrini (2011), Fernando Alves (2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020)

Question: List all chairpersons of Social Democrats, also known as Socialdemokratiet, from 2010 to 2020
Generated Answer: Yuriy Sánchez (2010, 2011, 2012), Igor Stasi (2010), Stefano Corella (2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020)

Question: List all positions Jyrki Katainen, also known as Jyrki Tapani Katainen, held from 2010 to 2020.
Generated Answer: Minister of Foreign Affairs (2010, 2011, 2012), Prime Minister of Finland (2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020)

Question: List all employers Ka

Question: List all employers Matt Ouimet, also known as Matthew Anthony Ouimet, worked for from 2010 to 2020.
Generated Answer: University of Queensland (2010, 2011, 2012), Australian National Institute for Science and Technology (2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020)

Question: List all employers Hunter Biden, also known as Robert Hunter Biden, worked for from 2013 to 2020.
Generated Answer: United States Senate (2013, 2014), Republican Party of the Democratic Caucus (2014, 2015, 2016, 2017, 2018, 2019, 2020)

Question: List all sports teams Adil Rami played for from 2010 to 2020.
Generated Answer: India national cricket team (2010, 2011, 2012), Mumbai Indians (2010), Chennai Super Kings (2011), Rajasthan Royals (2012), Delhi Daredevils (2013), Kolkata Knight Riders (2014), Sunrisers Hyderabad (2015), Sydney Sixers (2016)

Question: List all chairpersons of Cabinet of Ministers of Ukraine, also known as Government of Ukraine, from 2010 to 2020
Generated Answer: Kyivy T

Question: List all sports teams Lomana LuaLua played for from 2010 to 2020.
Generated Answer: FC Bayern Munich (2010, 2011, 2012), Real Madrid CF (2010), Juventus FC (2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020)

Question: List all entities that owned Culpeper Star-Exponent, also known as Culpeper Star Exponent, from 2015 to 2020.
Generated Answer: CPX Media Group (2015, 2016, 2017, 2018, 2019, 2020), Star-Express (2017, 2018)

Question: List all sports teams Aiden McGeady, also known as Aiden John McGeady, played for from 2010 to 2020.
Generated Answer: England national under-21 football team (2010), Manchester City F.C. (2010, 2011, 2012), Tottenham Hotspur A.F.C. (2012), West Bromwich Albion F.C. (2013), Stoke City F.C. (2014), Burnley F.C. (2015), Shrewsbury Town F.C. (2016), Blackpool MK (2017, 2018, 2019)

Question: List all coaches of NK Osijek, also known as Nogometni klub Osijek, from 2012 to 2020
Generated Answer: eljko Svijanovi (2012, 2013, 2014), Igor Kostu (2014,

Question: List all positions Grete Faremo held from 2010 to 2013.
Generated Answer: Minister of Foreign Affairs (2010, 2011, 2012), Prime Minister of Greece (2012, 2013)

Question: List all heads of the government of Rome, also known as The Eternal City, from 2010 to 2020 
Generated Answer: Giuseppe Inzaghi (2010, 2011, 2012), Piero Spalletti (2010), Luca Stimpelli (2011), Stefano Pioli (2012), Paolo Scone (2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020)

Question: List all heads of the government of North West, also known as North-West, from 2010 to 2020 
Generated Answer: David Cameron (2010, 2011, 2012), Tony Abbott (2010), Michael Morrison (2011), Peter Sellers (2012), Pauline Walker (2013), Jol Graaf (2014), Simon Grayson (2015)

Question: List all political parties Kyriakos Velopoulos, also known as Kyriakos Iosif Velopoulos, was a member of from 2010 to 2020.
Generated Answer: Democratic Convergence of Catalonia (2010, 2011, 2012), The People of Freedom (2010), Catalan Eur

Question: List all sports teams Cristiano Ronaldo, also known as Cristiano Ronaldo dos Santos Aveiro, played for from 2010 to 2020.
Generated Answer: Brazil national football team (2010, 2011, 2012), FC Barcelona (2010), Chelsea F.C. (2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020)

Question: List all positions Jo Johnson, also known as Joseph Johnson, held from 2018 to 2018.
Generated Answer: United States representative (2018, 2019), Governor of New York (2019)

Question: List all sports teams Rodrigo Palacio, also known as Rodrigo Sebastián Palacio Alcalde, played for from 2010 to 2020.
Generated Answer: FC Barcelona (2010, 2011, 2012), Real Madrid CF (2010), Chelsea F.C. (2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020)

Question: List all coaches of F.C. Nantes, also known as FC Nantes, from 2016 to 2020
Generated Answer: Jean-Claude Juncker (2016, 2017), Antoine Debussy (2017, 2018, 2019), Marcelo Arajo (2018), Aleksandr Sánchez (2019, 2020)

Question: List all s

In [32]:
#save predictions_3shot
with open('3shot_answers.json', 'w', encoding='utf-8') as f:
    json.dump(predictions_3shot, f, ensure_ascii=False, indent=4)

print("Answers saved to 3shot_answers.json")

Answers saved to 3shot_answers.json


In [33]:
examples_5shot = read_QA_data('examples_5shot.json')
predictions_5shot = run_model(last_checkpoint, test_questions, examples_5shot)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Question: List all sports teams Anthony Grant, also known as Anthony Paul Shaun Andrew Daure Grant, played for from 2010 to 2020.
Generated Answer: England national under-21 football team (2010), Manchester City F.C. (2010, 2011, 2012), Tottenham Hotspur A.F.C. (2012), West Bromwich Albion F.C. (2013), Stoke City F.C. (2014), Fulham F.C. (2015), Burnley F.C. (2016), Crystal Palace FC (2017, 2018), Queens Park Rangers F.C. (2018), Swindon Town F.C. (2018,

Question: List all positions Oleksandr Turchynov, also known as Oleksandr Valentynovych Turchynov, held from 2010 to 2020.
Generated Answer: President of Ukraine (2010, 2011, 2012), Prime Minister of Ukraine (2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020)

Question: List all political parties Paul Murphy was a member of from 2010 to 2020.
Generated Answer: Democratic Party (2010, 2011, 2012), independent politician (2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020)

Question: List all sports teams Andy Carroll, also known a

Question: List all sports teams Sam Tomkins played for from 2010 to 2020.
Generated Answer: England national under-21 football team (2010), Manchester City F.C. (2010, 2011, 2012), Tottenham Hotspur A.F.C. (2012), West Bromwich Albion F.C. (2013), Stoke City F.C. (2014), Fulham F.C. (2015), Burnley F.C. (2016), Crystal Palace FC (2017, 2018, 2019), Swindon Town F.C. (2018)

Question: List all sports teams Juan Cuadrado, also known as Juan Guillermo Cuadrado Bello, played for from 2010 to 2020.
Generated Answer: Real Madrid CF (2010, 2011, 2012), Chelsea F.C. (2010), Manchester City F.C. (2012, 2013), FC Bayern Munich (2013)

Question: List all political parties Rachel Carling-Jenkins, also known as Rachel Carling, was a member of from 2010 to 2018.
Generated Answer: Conservative Party (2010, 2011, 2012), Liberal Democrats (2010), National Alliance (2012, 2013, 2014, 2015, 2016, 2017, 2018)

Question: List all coaches of CD Tondela from 2015 to 2020
Generated Answer: tefan Stasi (2015, 

Question: List all positions François Lecointre, also known as General François Lecointre, held from 2016 to 2020.
Generated Answer: President of the French Republic (2016, 2017, 2018, 2019, 2020), Prime Minister of France (2017, 2018)

Question: List all coaches of Vitória S.C., also known as Vitória de Guimâraes, from 2015 to 2020
Generated Answer: José Mário (2015, 2016, 2017, 2018, 2019), Luis Martn Sánchez (2019, 2020)

Question: List all coaches of U.C. Trevigiani Energiapura Marchiol, also known as Unieuro Trevigiani-Hemus 1896, from 2012 to 2020
Generated Answer: A: Luis Enrique (2010, 2011, 2012), José Mourinho (2010), Juan Manuel Pellegrini (2011), Fernando Torres (2013), Antonio Conte (2014), Marco Silva (2015)

Question: List all coaches of Silkeborg IF, also known as Silkeborg Idrætsforening, from 2015 to 2020
Generated Answer: Erik Srensen (2015, 2016, 2017, 2018, 2019, 2020), Jörg Stahl (2020)

Question: List all chairpersons of Roman Catholic Archdiocese of Cardiff, als

Question: List all heads of the government of Guinea, also known as Guinea-Conakry, from 2010 to 2020 
Generated Answer: Emmanuel N'Ali (2010, 2011, 2012), Ousmane Mugabe (2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020)

Question: List all employers Linda Bauld, also known as Linda C. Bauld, worked for from 2014 to 2020.
Generated Answer: University of California, Irvine (2014, 2015), Stanford University (2015, 2016, 2017, 2018, 2019, 2020)

Question: List all coaches of PSV Eindhoven, also known as Philips Sport Vereniging, from 2013 to 2020
Generated Answer: Mauricio Pochettino (2013, 2014), Marcos Sánchez-Castro (2013), Marcelo De Oliveira (2018), Luis Enrique (2019, 2020)

Question: List all employers Pedro J. Ramírez worked for from 2010 to 2020.
Generated Answer: University of California, Los Angeles (2010, 2011, 2012), Universidad Nacional de Investigaciones Sociales Catalunyas (2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020)

Question: List all positions Charles Fal

...

Question: List all employers Kaywin Feldman worked for from 2010 to 2020.
Generated Answer: University of California, Irvine (2010, 2011, 2012), Stanford University (2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020)

Question: List all sports teams Emily Sonnett played for from 2012 to 2020.
Generated Answer: England national under-17 football team (2012, 2013, 2014), Manchester City F.C. (2014, 2015, 2016, 2017, 2018, 2019, 2020)

Question: List all positions Vera Baird, also known as Vera Thomas, held from 2010 to 2020.
Generated Answer: United States representative (2010, 2011, 2012), Governor of New York (2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020)

Question: List all employers Rémi Anicotte worked for from 2010 to 2014.
Generated Answer: University of California, Berkeley (2010, 2011, 2012), Université de Montréal (2012, 2013, 2014)

Question: List all positions Jean-Claude Juncker held from 2010 to 2019.
Generated Answer: European Commissioner for Economic Affairs (

Question: List all chairpersons of TOP 09, also known as Tradition Responsibility Prosperity, from 2010 to 2020
Generated Answer: Yuriy Kostunica (2010, 2011, 2012), Igor Sánchez-Sanchez (2010), Luis Antonio González (2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020)

Question: List all positions Mike Gallagher, also known as Michael John Gallagher, held from 2015 to 2020.
Generated Answer: Minister for Health (2015, 2016, 2017, 2018, 2019), Leader of the Opposition (2019, 2020)

Question: List all sports teams Javier Mascherano, also known as Javier Alejandro Mascherano, played for from 2010 to 2020.
Generated Answer: Arsenal F.C. (2010, 2011, 2012), Manchester City F.C. (2010), Chelsea F.C. (2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020)

Question: List all employers SethBling worked for from 2010 to 2020.
Generated Answer: University of California, Berkeley (2010, 2011, 2012), Stanford University (2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020)

Question: List all e

Question: List all educational institutions Clayton Custer attended from 2010 to 2020.
Generated Answer: University of Kentucky (2010, 2011, 2012), Texas A&M University (2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019)

Question: List all positions Mohamed Ghannouchi held from 2010 to 2011.
Generated Answer: President of Somalia (2010, 2011), Prime Minister of Somalia (2010)

Question: List all sports teams Jamal Crawford, also known as Aaron Jamal Crawford, played for from 2017 to 2020.
Generated Answer: Washington Nationals (2017, 2018, 2019), Los Angeles Lakers (2019, 2020)

Question: List all sports teams Humam Tariq, also known as Humam Tariq Faraj Naoush, played for from 2010 to 2020.
Generated Answer: Iraq national under-20 football team (2010), Al-Shorta Baghdad (2010, 2011, 2012), FC Twente (2011), Al Nasr SC (2012, 2013), Saudi Arabia national association football team (2012), AFC Bournemouth (2013, 2014, 2015, 2016, 2017, 2018, 2019)

Question: List all employers Julia Y Chan

Question: List all sports teams Shane Long, also known as Shane Patrick Long, played for from 2010 to 2020.
Generated Answer: England national under-21 football team (2010), Manchester United F.C. (2010, 2011, 2012), Tottenham Hotspur A.F.C. (2012), West Bromwich Albion F.C. (2013), Stoke City F.C. (2014), Nottingham Forest FC (2015)

Question: List all sports teams Filipe Morais, also known as Filipe Alexandre Major Morais, played for from 2010 to 2020.
Generated Answer: A: FC Barcelona (2010, 2011, 2012), Chelsea F.C. (2010), Real Madrid CF (2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020)

Question: List all political parties Marina Weisband was a member of from 2010 to 2020.
Generated Answer: Democratic Party (2010, 2011, 2012), Alternative for Germany (2010), Popular Unity (2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020)

Question: List all coaches of Spain women's national association football team, also known as Spain women's national soccer team, from 2010 to 2020
Ge

Question: List all coaches of F.C. Nantes, also known as FC Nantes, from 2016 to 2020
Generated Answer: Jean-Claude Juncker (2016, 2017), Antoine Debussy (2017, 2018, 2019), Marcelo Arajo (2018), Alfonso Sánchez (2019, 2020)

Question: List all sports teams Rafael van der Vaart, also known as Rafael Ferdinand van der Vaart, played for from 2010 to 2020.
Generated Answer: A: Manchester City F.C. (2010, 2011, 2012), Chelsea F.C. (2010), England national association football team (2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019), Tottenham Hotspur F.C. (2019, 2020)

Question: List all coaches of Inteja-Dominican, also known as Inteja-MMR Dominican, from 2015 to 2020
Generated Answer: José Luis Rodrguez González (2015, 2016, 2017, 2018, 2019), Juan Carlos A. Garca (2019, 2020)

Question: List all sports teams Jonny Howson, also known as Jonathan Mark Howson, played for from 2010 to 2020.
Generated Answer: England national under-21 football team (2010), Manchester City F.C. (2010, 2011

In [34]:
with open('5shot_answers.json', 'w', encoding='utf-8') as f:
    json.dump(predictions_5shot, f, ensure_ascii=False, indent=4)

print("Answers saved to 5shot_answers.json")

Answers saved to 5shot_answers.json


In [35]:
examples_7shot = read_QA_data('examples_7shot.json')
predictions_7shot = run_model(last_checkpoint, test_questions, examples_7shot)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Question: List all sports teams Anthony Grant, also known as Anthony Paul Shaun Andrew Daure Grant, played for from 2010 to 2020.
Generated Answer: England national under-19 football team (2010), Manchester United F.C. (2010, 2011, 2012), West Bromwich Albion A.F.C. (2012), Stoke City F.C. (2013), Liverpool F.C. (2014), Tottenham Hotspur F.C. (2015), Crystal Palace FC (2016, 2017), Burnley F.C. (2018)

Question: List all positions Oleksandr Turchynov, also known as Oleksandr Valentynovych Turchynov, held from 2010 to 2020.
Generated Answer: President of Serbia (2010, 2011, 2012), Prime Minister of Serbia (2012, 2013, 2014, 2015, 2016, 2017, 2018)

Question: List all political parties Paul Murphy was a member of from 2010 to 2020.
Generated Answer: Democratic Party (2010, 2011, 2012), independent politician (2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020)

Question: List all sports teams Andy Carroll, also known as Andrew Thomas Carroll, played for from 2010 to 2020.
Generated Ans

Question: List all sports teams Juan Cuadrado, also known as Juan Guillermo Cuadrado Bello, played for from 2010 to 2020.
Generated Answer: Real Madrid CF (2010, 2011, 2012), Chelsea F.C. (2010), Manchester City F.C. (2012, 2013), FC Bayern Munich (2013)

Question: List all political parties Rachel Carling-Jenkins, also known as Rachel Carling, was a member of from 2010 to 2018.
Generated Answer: Conservative Party (2010, 2011, 2012), Liberal Democrats (2010), National Alliance (2012, 2013, 2014, 2015, 2016, 2017, 2018)

Question: List all coaches of CD Tondela from 2015 to 2020
Generated Answer: tefan Stasi (2015, 2016, 2017, 2018, 2019), Ion Rafuț (2018), Mihail Petreșanev (2019, 2020)

Question: List all positions Brian Cowen, also known as Brian Bernard Cowen, held from 2010 to 2011.
Generated Answer: Minister for Health (2010, 2011), Leader of the Opposition (2010)

Question: List all political parties Ovidiu Silaghi, also known as Ovidiu Ioan Silaghi, was a member of from 2010 to

Question: List all coaches of U.C. Trevigiani Energiapura Marchiol, also known as Unieuro Trevigiani-Hemus 1896, from 2012 to 2020
Generated Answer: A: Luis Enrique (2010, 2011, 2012), José Mourinho (2010), Juan Carlos Garca Rodrguez González (2011), Fernando Torres (2013), Alberto Correa (2014), Marco Silva (2016), Maurizio Spalletti (2015)

Question: List all coaches of Silkeborg IF, also known as Silkeborg Idrætsforening, from 2015 to 2020
Generated Answer: Erik Srensen (2015, 2016, 2017, 2018, 2019), Jörg Storck (2019, 2020)

Question: List all chairpersons of Roman Catholic Archdiocese of Cardiff, also known as Archdiocese of Cardiff, from 2010 to 2020
Generated Answer: Peter O'Neill (2010, 2011, 2012), Michael McLeish (2010), Pauline Hendry (2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020)

Question: List all coaches of Qhubeka Assos, also known as Dimension Data, from 2015 to 2020
Generated Answer: Stefano Spalletti (2010, 2011, 2012), Sergei Svyazlovich (2012, 2013, 2014, 

Question: List all employers Linda Bauld, also known as Linda C. Bauld, worked for from 2014 to 2020.
Generated Answer: University of California, Irvine (2014, 2015), Stanford University (2015, 2016, 2017, 2018, 2019, 2020)

Question: List all coaches of PSV Eindhoven, also known as Philips Sport Vereniging, from 2013 to 2020
Generated Answer: Mauricio Pochettino (2013, 2014), Marcos Sánchez-Castro (2013), Marcelo De Oliveira (2018), Luis Enrique (2019, 2020)

Question: List all employers Pedro J. Ramírez worked for from 2010 to 2020.
Generated Answer: University of California, Berkeley (2010, 2011, 2012), Universidad Nacional de Investigaciones Sociales Catalunyas (2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020)

Question: List all positions Charles Falconer, Baron Falconer of Thoroton, also known as Charles Falconer, held from 2015 to 2020.
Generated Answer: Secretary of State for Education (2015, 2016, 2017, 2018, 2019), Shadow Home Office Deputy Prime Minister of England and 

...

Question: List all positions Vera Baird, also known as Vera Thomas, held from 2010 to 2020.
Generated Answer: United States representative (2010, 2011, 2012), Governor of New York (2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020)

Question: List all employers Rémi Anicotte worked for from 2010 to 2014.
Generated Answer: University of California, Berkeley (2010, 2011, 2012), Université de Montréal (2012, 2013, 2014)

Question: List all positions Jean-Claude Juncker held from 2010 to 2019.
Generated Answer: European Commissioner for Economic Affairs (2010, 2011, 2012), President of the Council of Ministers of Europe (2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019)

Question: List all positions Benny Gantz, also known as Benjamin Gantz, held from 2011 to 2020.
Generated Answer: United States representative (2011, 2012, 2013, 2014), Governor of New Hampshire (2014, 2015, 2016, 2017, 2018, 2019, 2020)

Question: List all coaches of Gaziantep F.K. from 2013 to 2020
Generated Answer: Yur

Question: List all sports teams Javier Mascherano, also known as Javier Alejandro Mascherano, played for from 2010 to 2020.
Generated Answer: Netherlands national association football team (2010, 2011, 2012), FC Bayern Munich (2010), S.S.C. Napoli (2011, 2012), AFC Bournemouth (2012, 2013)

Question: List all employers SethBling worked for from 2010 to 2020.
Generated Answer: University of California, Berkeley (2010, 2011, 2012), Stanford University (2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020)

Question: List all entities that owned OneWeb, also known as One Web, from 2010 to 2020.
Generated Answer: Google (2010, 2011, 2012), Yahoo! Inc. (2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020)

Question: List all sports teams Roda Antar played for from 2010 to 2016.
Generated Answer: Arsenal F.C. (2010, 2011, 2012), Manchester City F.C. (2010), Chelsea F.C. (2012, 2013, 2014, 2015, 2016)

Question: List all sports teams Edwin van der Sar, also known as Edwin van der Saar, playe

Question: List all sports teams Jamal Crawford, also known as Aaron Jamal Crawford, played for from 2017 to 2020.
Generated Answer: Washington Nationals (2017, 2018, 2019), Los Angeles Lakers (2019, 2020)

Question: List all sports teams Humam Tariq, also known as Humam Tariq Faraj Naoush, played for from 2010 to 2020.
Generated Answer: Iraq national under-20 football team (2010), Al-Shorta Baghdad (2010, 2011, 2012), FC Twente (2011), Al Nasr SC (2012, 2013), Saudi Arabia national association football team (2012), AFC Bournemouth (2013, 2014, 2015, 2016, 2017, 2018, 2019)

Question: List all employers Julia Y Chan, also known as Julia Y. Chan, worked for from 2010 to 2020.
Generated Answer: University of California, Berkeley (2010, 2011, 2012), Stanford University (2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020)

Question: List all sports teams Peter Crouch, also known as Peter James Crouch, played for from 2010 to 2020.
Generated Answer: Arsenal F.C. (2010, 2011, 2012), England

Question: List all sports teams Shane Long, also known as Shane Patrick Long, played for from 2010 to 2020.
Generated Answer: England national under-21 football team (2010), Manchester City F.C. (2010, 2011, 2012), Aston Villa F.C. (2012), West Bromwich Albion F.C. (2013), Shrewsbury Town F.C. (2014)

Question: List all sports teams Filipe Morais, also known as Filipe Alexandre Major Morais, played for from 2010 to 2020.
Generated Answer: A: England national under-21 football team (2010), Manchester United F.C. (2010, 2011, 2012), Chelsea F.C. (2012), Tottenham Hotspur F.C. (2013), Liverpool F.C. (2014), West Bromwich Albion F.C. (2015), Stoke City F.C. (2016), Burnley F.C. (2018), Crystal Palace FC (2019, 2020)

Question: List all political parties Marina Weisband was a member of from 2010 to 2020.
Generated Answer: Democratic Party (2010, 2011, 2012), Alternative for Germany (2010), Popular Unity (2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020)

Question: List all coaches of Sp

Question: List all coaches of F.C. Nantes, also known as FC Nantes, from 2016 to 2020
Generated Answer: Jean-Claude Juncker (2016, 2017), Antoine Deschamps (2018), Marcelo Inzaghi (2018, 2019), Alain Sánchez (2019, 2020)

Question: List all sports teams Rafael van der Vaart, also known as Rafael Ferdinand van der Vaart, played for from 2010 to 2020.
Generated Answer: A: Manchester City F.C. (2010, 2011, 2012), Chelsea F.C. (2010), England national association football team (2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019), Tottenham Hotspur F.C. (2019, 2020)

Question: List all coaches of Inteja-Dominican, also known as Inteja-MMR Dominican, from 2015 to 2020
Generated Answer: Cristiano Ronaldo (2015, 2016, 2017, 2018, 2019), Luis Enrique (2016)

Question: List all sports teams Jonny Howson, also known as Jonathan Mark Howson, played for from 2010 to 2020.
Generated Answer: England national under-21 football team (2010), Manchester City F.C. (2010, 2011, 2012), Tottenham Hotspur A

In [36]:
with open('7shot_answers.json', 'w', encoding='utf-8') as f:
    json.dump(predictions_7shot, f, ensure_ascii=False, indent=4)
print("Answers saved to 7shot_answers.json")

Answers saved to 7shot_answers.json
